In [ ]:
# | default_exp classes.DomoDatacenter


In [ ]:
# | exporti
import asyncio
from dataclasses import dataclass
from typing import Union
from enum import Enum

import httpx

from fastcore.basics import patch_to

import domolibrary.utils.chunk_execution as ce

import domolibrary.client.DomoAuth as dmda
import domolibrary.classes.DomoDataset as dmds
import domolibrary.classes.DomoAccount as dma
import domolibrary.classes.DomoCard as dmc 

import domolibrary.routes.datacenter as datacenter_routes
import domolibrary.routes.card as card_routes


In [ ]:
# | export

@dataclass
class DomoDatacenter:
    "class for quering entities in the datacenter"
    auth: dmda.DomoAuth = None

In [ ]:
# | exporti

@patch_to(DomoDatacenter, cls_method=True)
async def search_datacenter(
    cls,
    auth: dmda.DomoAuth,
    maximum: int = None,  # maximum number of results to return
    body: dict = None,  # either pass a body or generate a body in the function using search_text, entity_type, and additional_filters parameters
    search_text=None,
    # can accept one value or a list of values
    entity_type: Union[str, list] = "dataset",
    additional_filters_ls=None,
    return_raw: bool = False,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
) -> list:

    res = await datacenter_routes.search_datacenter(
        auth=auth,
        maximum=maximum,
        body=body,
        session=session,
        search_text=search_text,
        entity_type=entity_type,
        additional_filters_ls=additional_filters_ls,
        debug_api=debug_api,
    )

    if return_raw:
        return res

    return res.response

#### sample implementation of search_datacenter


In [ ]:
import domolibrary.client.DomoAuth as dmda
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

additional_filters_ls = [
    datacenter_routes.generate_search_datacenter_filter(
        "dataprovidername_facet", "Jupyter Data"
    )
]

res = await DomoDatacenter.search_datacenter(
    auth=token_auth,
    additional_filters_ls=additional_filters_ls,
       search_text = "*kb*",
    entity_type=datacenter_routes.Datacenter_Enum.DATASET.value,
)

pd.DataFrame(res[0:5])

,entityType,databaseId,searchId,createDate,lastModified,lastIndexed,highlightedFields,language,requestAccess,score,...,pdpEnabled,used,owners,ownersLocalized,cloudName,cloudId,winnerText,systemTagFields,ownedByType,customer
0,dataset,04c1574e-c8be-4721-9846-c6ffa491144b,"{'indexName': None, 'databaseId': '04c1574e-c8...",1668379680000,1695306445000,1695314768905,{},English,False,11.0,...,False,False,"[{'id': '1893952720', 'type': 'USER', 'display...","{'localizedMessage': 'Jae Wilson1', 'count': 1}",Domo,domo,domo_kbs,[system_tags],USER,mmmm-0012-0200


In [ ]:
import domolibrary.client.DomoAuth as dmda
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)


res = await DomoDatacenter.search_datacenter(
    auth=token_auth,
    entity_type=datacenter_routes.Datacenter_Enum.CARD.value,
)

pd.DataFrame(res[0:5])


In [ ]:
# | exporti
@patch_to(DomoDatacenter, cls_method=True)
async def search_datasets(
    cls,
    auth=dmda.DomoAuth,
    maximum: int = None,  # maximum number of results to return
    search_text=None,
    # can accept one value or a list of values
    additional_filters_ls=None,
    return_raw: bool = False,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
) -> list[dmds.DomoDataset]:

    json_list = await cls.search_datacenter(
        auth=auth,
        maximum=maximum,
        search_text=search_text,
        entity_type=datacenter_routes.Datacenter_Enum.DATASET.value,
        additional_filters_ls=additional_filters_ls,
        return_raw=return_raw,
        session=session,
        debug_api=debug_api,
    )

    if return_raw or len(json_list) == 0:
        return json_list

    return await ce.gather_with_concurrency( n = 60,
        *[
            dmds.DomoDataset.get_from_id(
                dataset_id=json_obj.get("databaseId"),
                auth=auth,
                debug_api=debug_api,
                session=session,
            )
            for json_obj in json_list
        ]
    )

#### sample implementation search_datasets


In [ ]:
import domolibrary.client.DomoAuth as dmda
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

additional_filters_ls = [
    datacenter_routes.generate_search_datacenter_filter(
        "dataprovidername_facet", "Jupyter Data"
    )
]

res = await DomoDatacenter.search_datasets(
    auth=token_auth,
    search_text="*kb*",
    additional_filters_ls=additional_filters_ls,
)

res

[DomoDataset(id='04c1574e-c8be-4721-9846-c6ffa491144b', display_type='domo-jupyterdata', data_provider_type='domo-jupyterdata', name='domo_kbs', description=None, row_count=1185, column_count=7, stream_id=825, owner={'id': '1893952720', 'name': 'Jae Wilson1', 'type': 'USER', 'group': False}, formula={'calculation_ca9d4b1c-f73a-4f76-9f94-d3c4ca6871c5': {'templateId': 2664, 'id': 'calculation_ca9d4b1c-f73a-4f76-9f94-d3c4ca6871c5', 'name': 'rowcount', 'formula': 'sum(1)', 'status': 'VALID', 'dataType': 'LONG', 'persistedOnDataSource': True, 'isAggregatable': True, 'bignumber': False}, 'calculation_38846559-d190-4ab1-809b-bcd361db5670': {'templateId': 2665, 'id': 'calculation_38846559-d190-4ab1-809b-bcd361db5670', 'name': 'max_views', 'formula': 'max(views)', 'status': 'VALID', 'dataType': 'LONG', 'persistedOnDataSource': True, 'isAggregatable': True, 'bignumber': False, 'columnPositions': [{'columnName': 'views', 'columnPosition': 4}]}}, schema=DomoDataset_Schema(dataset=..., columns=[]),

In [ ]:
# | exporti
@patch_to(DomoDatacenter, cls_method=True)
async def get_accounts(
    cls,
    auth=dmda.DomoAuth,
    maximum: int = None,  # maximum number of results to return
    # can accept one value or a list of values
    additional_filters_ls=None,
    return_raw: bool = False,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
) -> list[dma.DomoAccount]:
    """search Domo Datacenter account api.
Note: at the time of this writing 7/18/2023, the datacenter api does not support searching accounts by name"""
    
    json_list = await cls.search_datacenter(
        auth=auth,
        maximum=maximum,
        entity_type=datacenter_routes.Datacenter_Enum.ACCOUNT.value,
        additional_filters_ls=additional_filters_ls,
        return_raw=return_raw,
        session=session,
        debug_api=debug_api,
    )

    if return_raw or len(json_list) == 0:
        return json_list

    domo_account_ls = [
            dma.DomoAccount._from_json(
                json_obj,
                auth=auth ) for json_obj in json_list
        ]
    
    return domo_account_ls

#### sample implementation search_accounts


In [ ]:
import domolibrary.client.DomoAuth as dmda
import domolibrary.routes.datacenter as datacenter_routes
import os
import pandas as pd

auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

additional_filters_ls = [
    datacenter_routes.generate_search_datacenter_filter(
        datacenter_routes.Datacenter_Filter_Field_Enum.DATAPROVIDER.value,
        "Google Sheets",
    )
]

res = await DomoDatacenter.get_accounts(
    auth=auth,
    # additional_filters_ls=additional_filters_ls,
    return_raw=False,
    debug_api=False,
)

print(len(res))
res[0:5]

51


[DomoAccount(name='**ReplicationUtility**:modocorp.domo.com', data_provider_type='dataset-copy', id='50', created_dt=datetime.datetime(2022, 10, 6, 19, 20, 34), modified_dt=datetime.datetime(2022, 10, 6, 19, 20, 34), config=None),
 DomoAccount(name='onyxReporting@gmail.com', data_provider_type='google-spreadsheets', id='45', created_dt=datetime.datetime(2022, 10, 4, 22, 59, 14), modified_dt=datetime.datetime(2022, 10, 4, 22, 59, 14), config=None),
 DomoAccount(name='Domo Governance Datasets Third Party Account', data_provider_type='domo-governance-d14c2fef-49a8-4898-8ddd-f64998005600', id='5', created_dt=datetime.datetime(2021, 3, 26, 16, 54, 41), modified_dt=datetime.datetime(2021, 3, 26, 16, 54, 41), config=None),
 DomoAccount(name='jaemyong.wilson@sony.com', data_provider_type='google-spreadsheets', id='70', created_dt=datetime.datetime(2023, 4, 26, 12, 20, 51), modified_dt=datetime.datetime(2023, 4, 26, 12, 20, 51), config=None),
 DomoAccount(name='domo_creds', data_provider_type='

In [ ]:
import domolibrary.classes.DomoDataset as dmds

a = dmds.DomoDataset(id = 123)
a.__class__.__name__


datacenter_routes.Datacenter_Enum.DATAFLOW.value

'DATAFLOW'

In [ ]:
#| exporti

class LineageTypes_Enum(Enum):
    DomoDataset = 'DATA_SOURCE'
    DomoDataflow = 'DATAFLOW'


@patch_to(DomoDatacenter, cls_method=True)
async def get_lineage_upstream(
    cls,
    auth: dmda.DomoAuth,
    domo_entity, # DomoDataset or DomoDataflow
    return_raw: bool = False,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
    debug_prn: bool = False):

    import domolibrary.classes.DomoDataset as dmds
    import domolibrary.classes.DomoDataflow as dmdf

    if not session:
        session = httpx.AsyncClient()
        is_close_session = True


    res = await datacenter_routes.get_lineage_upstream(
        auth=auth,
        entity_type= LineageTypes_Enum[domo_entity.__class__.__name__].value,
        entity_id=domo_entity.id,
        session=session,
        debug_api=debug_api,
    )

    if return_raw or res.status != 200:
        await session.aclose()
        return res

    obj = res.response

    domo_obj = []
    for key, item in obj.items():
        if item.get("type") == "DATA_SOURCE":
            domo_obj.append(
                await dmds.DomoDataset.get_from_id(auth=auth, dataset_id=item.get("id"), session= session)
            )

        if item.get("type") == "DATAFLOW":
            # print(item.get('id'))
            domo_obj.append(
                await dmdf.DomoDataflow.get_from_id(
                    auth=auth, dataflow_id=item.get("id"), session = session
                    
                )
            )
            pass

    await session.aclose()
    return domo_obj
    

In [ ]:
import domolibrary.client.DomoAuth as dmda
import domolibrary.classes.DomoDataset as dmds
import os

auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_dataset = dmds.DomoDataset(id = '45a7a24e-c738-44f9-9019-00d5a33613ae')

await DomoDatacenter.get_lineage_upstream(
    auth=auth,
    domo_entity = domo_dataset
    # return_raw = True,
    # debug_api = True
)


[DomoDataset(id='45a7a24e-c738-44f9-9019-00d5a33613ae', display_type='dataflow', data_provider_type='dataflow', name='sys_date', description=None, row_count=7299, column_count=9, stream_id=None, owner={'id': '1893952720', 'name': 'Jae Wilson1', 'type': 'USER', 'group': False}, formula={}, schema=DomoDataset_Schema(dataset=..., columns=[]), tags=DomoDataset_Tags(dataset=..., tag_ls=['s_domostats', 'domoStats', 'domostats', 'c_people']), certification=None, PDP=<domolibrary.classes.DomoPDP.Dataset_PDP_Policies object>),
 DomoDataflow(id=2, name='sys_numbers', auth=DomoTokenAuth(domo_instance='domo-community', token_name='token_auth', is_valid_token=True, url_manual_login='https://domo-community.domo.com/auth/index?domoManualLogin=true'), owner=None, description=None, tags=None, actions=[DomoDataflow_Action(type='LoadFromVault', id='241025d7-3cca-4369-b7c0-b3264277c0e1', name='people', data_source_id='241025d7-3cca-4369-b7c0-b3264277c0e1', sql=None), DomoDataflow_Action(type='GenerateTabl

In [ ]:
# | exporti
@patch_to(DomoDatacenter, cls_method=True)
async def search_cards(
    cls,
    auth=dmda.DomoAuth,
    maximum: int = None,  # maximum number of results to return
    search_text=None,
    # can accept one value or a list of values
    additional_filters_ls=None,
    return_raw: bool = False,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
) -> list[dmds.DomoDataset]:

    json_list = await cls.search_datacenter(
        auth=auth,
        maximum=maximum,
        search_text=search_text,
        entity_type=datacenter_routes.Datacenter_Enum.CARD.value,
        additional_filters_ls=additional_filters_ls,
        return_raw=return_raw,
        session=session,
        debug_api=debug_api,
    )

    if return_raw or len(json_list) == 0:
        return json_list

    return await ce.gather_with_concurrency( n = 60,
        *[
            dmc.DomoCard.get_by_id(
                card_id=json_obj.get("databaseId"),
                auth=auth,
                debug_api=debug_api,
                session=session,
            )
            for json_obj in json_list
        ]
    )

In [ ]:
import os
import domolibrary.client.DomoAuth as dmda
from domolibrary.routes.datacenter import generate_search_datacenter_filter, Datacenter_Filter_Field_Enum, Datacenter_Filter_Field_Certification_Enum

auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

additional_filters_ls = [
    generate_search_datacenter_filter(
        field=Datacenter_Filter_Field_Enum.CERTIFICATION.value,
        value=Datacenter_Filter_Field_Certification_Enum.CERTIFIED.value
    )]

print(additional_filters_ls)

await DomoDatacenter.search_cards(
    auth=auth,
    additional_filters_ls = additional_filters_ls,
    maximum=5
    # return_raw = True,
    # debug_api = True
)


[{'filterType': 'term', 'field': 'certification.state', 'value': 'CERTIFIED', 'not': False}]


[DomoCard(id=1693394557, title='gov_datasetEfficiency', description='', type='kpi', urn='1693394557', chart_type='badge_basic_table', dataset_id='4fc29727-43a4-47ad-94a0-d0021dad8f40', certification=DictDot(state='CERTIFIED', lastUpdated=1690401760016, userId='1893952720', processType='COMPANY', approvalRequestId='80573acf-7877-44b1-b995-2e9b869a936d', processName='Company', adminCertified=False), owners=[DomoUser(id='1893952720', title=None, department='Business Improvement', display_name='Jae Wilson1', email_address='jae@onyxreporting.com', role_id=810756122, avatar_key='c605f478-0cd2-4451-9fd4-d82090b71e66', phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None)])]

In [ ]:
# | exporti

@patch_to(DomoDatacenter, cls_method=True)
async def get_cards_admin_summary(
    cls,
    auth=dmda.DomoAuth,
    page_ids: [str] = None,
    card_search_text: str = None,
    page_search_text: str = None,

    maximum: int = None,  # maximum number of results to return
    # can accept one value or a list of values
    return_raw: bool = False,
    debug_api: bool = False,
    debug_loop: bool = False,
    session: httpx.AsyncClient = None,
) -> list[dmc.DomoCard]:
    """search Domo Datacenter card api."""

    search_body = card_routes.generate_body_search_cards_admin_summary(page_ids = page_ids,
                                             card_search_text = card_search_text,
                                             page_search_text = page_search_text)

    res = await card_routes.search_cards_admin_summary(
        auth=auth,
        body = search_body,
        maximum=maximum,
        debug_api=debug_api,
        debug_loop=debug_loop,
        session=session,
        wait_sleep = 5
    )

    if return_raw or len(res.response) == 0:
        return res

    domo_account_ls = await ce.gather_with_concurrency(n=60,*[
            dmc.DomoCard._from_json(
                json_obj,
                auth=auth ) for json_obj in res.response
        ])

    return domo_account_ls


In [ ]:
import os
import domolibrary.client.DomoAuth as dmda

auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

await DomoDatacenter.get_cards_admin_summary(
    auth=auth,
    # return_raw = True,
    maximum = 5
    # debug_api = True
)


[DomoCard(id=332819959, title=' Forecast-Sales_MG', description=None, type='kpi', urn=None, chart_type=None, dataset_id=None, certification=None, owners=[DomoUser(id='722128535', title=None, department=None, display_name='Miguel Guedes', email_address='Miguel.guedes@cgi.com', role_id=2, avatar_key='ef7c051e-2b2f-4ad4-844b-bf348853beb6', phone_number='+351913017621', web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None)]),
 DomoCard(id=1582016299, title=" Incorrect Append - Forecast Column Header Names Don't Match", description=None, type='document', urn=None, chart_type=None, dataset_id=None, certification=None, owners=[DomoUser(id='583215149', title='Sr. Customer Experience Eng.', department='Domo Client Services', display_name='John Jarnagin', email_address='John.Jarnagin@domo.com', role_id=1, avatar_key='f672207a-b3b4-4142-9036-560086f4b297', p

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()